In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
tf.set_random_seed(777)

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [22]:
tf.reset_default_graph()

In [24]:
# hyperparameter
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [25]:
class Model :
    
    def __init__(self, sess, name) :
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self) :
        with tf.variable_scope(self.name) :
            # dropout(keep_prob) rate
            self.keep_prob = tf.placeholder(tf.float32)
            
            # input placeholders
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            # L1 Imgin shape = (?, 28, 28, 1)
            W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
            L1 = tf.nn.conv2d(X_img, W1, strides = [1, 1, 1, 1], padding = 'SAME')
            L1 = tf.nn.relu(L1)
            L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            L1 = tf.nn.dropout(L1, keep_prob = self.keep_prob)
            
            # L2 ImgIn shape = (?, 14, 14, 32)
            W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
            L2 = tf.nn.conv2d(L1, W2, strides = [1, 1, 1, 1], padding = 'SAME')
            L2 = tf.nn.relu(L2)
            L2 = tf.nn.max_pool(L2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            L2 = tf.nn.dropout(L2, keep_prob = self.keep_prob)
            
            # L3 ImgIn shape = (?, 7, 7, 64)
            W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev = 0.01))
            L3 = tf.nn.conv2d(L2, W3, strides = [1, 1, 1, 1], padding = 'SAME')
            L3 = tf.nn.relu(L3)
            L3 = tf.nn.max_pool(L3, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            L3 = tf.nn.dropout(L3, keep_prob = self.keep_prob)
            L3_flat = tf.reshape(L3, [-1, 128 * 4 * 4])
            
            # L4 FC 4 x 4 x 128 inputs -> 625 outputs
            W4 = tf.get_variable("W4", shape = [128 * 4 * 4, 625], initializer = tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.random_normal([625]))
            L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
            L4 = tf.nn.dropout(L4, keep_prob = self.keep_prob)
            
            # L5 Final FC 625 inputs -> 10 outputs
            W5 = tf.get_variable("W5", shape = [625, 10], initializer = tf.contrib.layers.xavier_initializer())
            b5 = tf.Variable(tf.random_normal([10]))
            self.logits = tf.matmul(L4, W5) + b5
            
        # define coslt/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
    
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    def train(self, x_data, y_data, keep_prob = 0.7) :
        return self.sess.run([self.cost, self.optimizer], feed_dict = {self.X : x_data, self.Y : y_data, self.keep_prob : keep_prob})
    
    def predict(self, x_test, keep_prob = 1.0) :
        return self.sess.run(self.logits, feed_dict = {self.X : x_test, self.keep_prob : keep_prob})
    
    def get_accuracy(self, x_test, y_test, keep_prob = 1.0) :
        return self.sess.run(self.accuracy, feed_dict = {self.X : x_test, self.Y : y_test, self.keep_prob : keep_prob})

In [26]:
# initialize
sess = tf.Session()
m1 = Model(sess, 'm1')

sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epochs) :
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch) :
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch
        
    print('Epoch : ', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost))
    
print('Learning Finished!')

# Test model and check accuracy
print('Accuracy : ', m1.get_accuracy(mnist.test.images, mnist.test.labels))

Learning Started!
Epoch :  0001 cost =  0.388648453
Epoch :  0002 cost =  0.098360058
Epoch :  0003 cost =  0.071898937
Epoch :  0004 cost =  0.061576256
Epoch :  0005 cost =  0.052592415
Epoch :  0006 cost =  0.046822920
Epoch :  0007 cost =  0.042230461
Epoch :  0008 cost =  0.040253814
Epoch :  0009 cost =  0.036886328
Epoch :  0010 cost =  0.034985219
Epoch :  0011 cost =  0.029932655
Epoch :  0012 cost =  0.029851733
Epoch :  0013 cost =  0.028402187
Epoch :  0014 cost =  0.027232494
Epoch :  0015 cost =  0.025473854
Learning Finished!
Accuracy :  0.9936
